In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

In [ ]:
import argparse

# Parsing command line arguments
parser = argparse.ArgumentParser(description='Object Detection')
parser.add_argument('--video', type=str, help='Path to video file')
args = parser.parse_args()

# Initializing video capture
cap = cv2.VideoCapture(args.video)


In [ ]:
# Loading YOLOv8 model
model = YOLO("yolov8n.pt")  

In [ ]:
# Defining ROI or line for counting
line_position = 300
car_count = 0
counted_ids = set()

In [ ]:
import cv2

def get_car_color(roi):
    # Convert ROI to HSV color space
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    
    # Define HSV color ranges
    color_ranges = {
        "Red": [(0, 50, 50), (10, 255, 255), (170, 50, 50), (180, 255, 255)],
        "Orange": [(10, 50, 50), (25, 255, 255)],
        "Yellow": [(25, 50, 50), (35, 255, 255)],
        "Green": [(35, 50, 50), (85, 255, 255)],
        "Blue": [(85, 50, 50), (125, 255, 255)],
        "Purple": [(125, 50, 50), (145, 255, 255)],
        "Pink": [(145, 50, 50), (170, 255, 255)],
        "White": [(0, 0, 200), (180, 30, 255)]
    }
    
    max_pixels = 0
    dominant_color = ""
    
    for color, ranges in color_ranges.items():
        mask = None
        for i in range(0, len(ranges), 2):
            lower_bound = np.array(ranges[i])
            upper_bound = np.array(ranges[i+1])
            if mask is None:
                mask = cv2.inRange(hsv, lower_bound, upper_bound)
            else:
                mask += cv2.inRange(hsv, lower_bound, upper_bound)
        
        num_pixels = cv2.countNonZero(mask)
        if num_pixels > max_pixels:
            max_pixels = num_pixels
            dominant_color = color

    # Applying inverse rule
    if dominant_color == "Red":
        return "Blue"
    elif dominant_color == "Blue":
        return "Red"
    
    return dominant_color

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (600, 400))
    height, width, channels = frame.shape

    # Detect cars using YOLOv8
    results = model.track(frame, persist=True)

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # Get bounding boxes
        confidences = result.boxes.conf.cpu().numpy()  # Get confidences
        class_ids = result.boxes.cls.cpu().numpy()  # Get class IDs
        # Check if track IDs are available
        if result.boxes.id is not None:
            track_ids = result.boxes.id.cpu().numpy()  # Get track IDs
        else:
            track_ids = [None] * len(boxes)  # Use None for track IDs if not available

        for i in range(len(boxes)):
            if confidences[i] > 0.5 and class_ids[i] == 2:  # Class ID 2 is for 'car' in COCO dataset
                x, y, x2, y2 = boxes[i]
                x, y, w, h = int(x), int(y), int(x2 - x), int(y2 - y)
                roi = frame[y:y+h, x:x+w]
                tid = track_ids[i]
                car_color = get_car_color(roi)
                label = f"{car_color}"
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Checking if the car crosses the line
                if y2 > line_position and y < line_position and tid not in counted_ids:
                    car_count += 1
                    counted_ids.add(tid)


    # Drawing counting line
    cv2.line(frame, (0, line_position), (width, line_position), (0, 0, 255), 2)
    cv2.putText(frame, f"Count: {car_count}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()